In [44]:
#downloaded data from http://www.manythings.org/anki/

In [45]:
import pandas as pd
import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model

In [46]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000

In [47]:
data = pd.read_csv('fra-eng/fra.txt', sep='\t',names=['english','french','comment'])
data.head()

,english,french,comment
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
4,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [48]:
eng_corpus = []
preprocessed_data = data[['english','french']][:1]
for i in range(len(data)):
    if not data['english'][i] in eng_corpus:
        eng_corpus.append(data['english'][i])
        preprocessed_data = preprocessed_data.append(data[['english','french']][i:i+1])
       
    if len(eng_corpus) > num_samples:
        break

In [49]:
len(preprocessed_data)

10002

In [50]:
preprocessed_data.reset_index(inplace=True, drop=True)

In [51]:
import string
import re
exclude = set(string.punctuation)

In [52]:
#preprocessed_data = data[['english','hindi']]

In [53]:
def preprocess_text(text):
    return '\t'+text.lower()+'\n'

In [54]:
eng_corpus = []
frc_corpus = []
for i in range(len(preprocessed_data)):
    
    eng_corpus.append(preprocess_text(preprocessed_data['english'][i]))
    frc_corpus.append(preprocess_text(preprocessed_data['french'][i]))

In [55]:
eng_vocab = []
frc_vocab = []
for i in range(len(preprocessed_data)):
    for word in eng_corpus[i]:
        if not word in eng_vocab:
            eng_vocab.append(word)
    for word in frc_corpus[i]:
        if word not in frc_vocab:
            frc_vocab.append(word)

In [56]:
print('length of english words: ', len(eng_vocab))
print('length of french words: ', len(frc_vocab))

length of english words:  51
length of french words:  69


In [57]:
max_input_length = max([len(words) for words in eng_corpus])
print("Max input length",max_input_length)
max_output_length = max([len(words) for words in frc_corpus])
print("Max output length",max_output_length)

Max input length 19
Max output length 59


In [58]:
input_words = sorted(eng_vocab)
output_words = sorted(frc_vocab)

num_encoder_tokens = len(eng_vocab)
num_decoder_tokens = len(frc_vocab)
print('encoder tokens: ', num_encoder_tokens)
print('decoder tokens: ', num_decoder_tokens)

encoder tokens:  51
decoder tokens:  69


In [59]:
input_token_index = dict([(i, char) for char, i in enumerate(input_words)])
target_token_index = dict([(i, char) for char, i in enumerate(output_words)])

In [60]:
encoder_input_data = np.zeros((len(eng_corpus),max_input_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(eng_corpus),max_output_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(eng_corpus),max_output_length, num_decoder_tokens),dtype='float32') 

In [61]:
for i, (input_text, target_text) in enumerate(zip(eng_corpus, frc_corpus)):
    
    for j, char in enumerate(input_text):
        encoder_input_data[i, j, input_token_index[char]] = 1.
    encoder_input_data[i, j+1:, input_token_index[' ']] = 1.
    
    for j, char in enumerate(target_text):
        decoder_input_data[i, j, target_token_index[char]] = 1.
        if j>0:
            decoder_target_data[i, j-1, target_token_index[char]] = 1.
    decoder_input_data[i, j+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, j:, target_token_index[' ']] = 1.
        

In [62]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10002, 19, 51)
(10002, 59, 69)
(10002, 59, 69)


In [63]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state= True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [64]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [65]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
126/126 [==============================] - 90s 678ms/step - loss: 1.1551 - accuracy: 0.7211 - val_loss: 1.0811 - val_accuracy: 0.6869
Epoch 2/100
126/126 [==============================] - 68s 540ms/step - loss: 0.8456 - accuracy: 0.7699 - val_loss: 0.8683 - val_accuracy: 0.7515
Epoch 3/100
126/126 [==============================] - 77s 613ms/step - loss: 0.6939 - accuracy: 0.8021 - val_loss: 0.8051 - val_accuracy: 0.7602
Epoch 4/100
126/126 [==============================] - 80s 637ms/step - loss: 0.6229 - accuracy: 0.8194 - val_loss: 0.8048 - val_accuracy: 0.7638
Epoch 5/100
126/126 [==============================] - 80s 634ms/step - loss: 0.5685 - accuracy: 0.8334 - val_loss: 0.6860 - val_accuracy: 0.7986
Epoch 6/100
126/126 [==============================] - 99s 787ms/step - loss: 0.5294 - accuracy: 0.8441 - val_loss: 0.6614 - val_accuracy: 0.8071
Epoch 7/100
126/126 [==============================] - 88s 700ms/step - loss: 0.5000 - accuracy: 0.8521 - val_loss: 0.6253 -

Epoch 57/100
126/126 [==============================] - 76s 603ms/step - loss: 0.1319 - accuracy: 0.9607 - val_loss: 0.5508 - val_accuracy: 0.8697
Epoch 58/100
126/126 [==============================] - 78s 620ms/step - loss: 0.1288 - accuracy: 0.9616 - val_loss: 0.5595 - val_accuracy: 0.8698
Epoch 59/100
126/126 [==============================] - 75s 598ms/step - loss: 0.1262 - accuracy: 0.9623 - val_loss: 0.5593 - val_accuracy: 0.8714
Epoch 60/100
126/126 [==============================] - 75s 592ms/step - loss: 0.1230 - accuracy: 0.9634 - val_loss: 0.5991 - val_accuracy: 0.8636
Epoch 61/100
126/126 [==============================] - 75s 597ms/step - loss: 0.1205 - accuracy: 0.9641 - val_loss: 0.5743 - val_accuracy: 0.8707
Epoch 62/100
126/126 [==============================] - 76s 607ms/step - loss: 0.1182 - accuracy: 0.9646 - val_loss: 0.5835 - val_accuracy: 0.8687
Epoch 63/100
126/126 [==============================] - 76s 604ms/step - loss: 0.1159 - accuracy: 0.9655 - val_loss: 0

In [66]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [67]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [68]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_output_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [69]:
for seq_index in range(10):
    
    input_seq = encoder_input_data[seq_index: seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('\n-')
    print('Input text:', eng_corpus[seq_index])
    print('Output text:', decoded_sentence)
    print("Actual text", preprocessed_data.french[seq_index: seq_index+1])
    


-
Input text: 	go.

Output text: vais-lui !

Actual text 0    Va !
Name: french, dtype: object

-
Input text: 	go.

Output text: vais-lui !

Actual text 1    Va !
Name: french, dtype: object

-
Input text: 	hi.

Output text: salut !

Actual text 2    Salut !
Name: french, dtype: object

-
Input text: 	run!

Output text: cours !

Actual text 3    Cours !
Name: french, dtype: object

-
Input text: 	run.

Output text: cours !

Actual text 4    Cours !
Name: french, dtype: object

-
Input text: 	who?

Output text: qui ?

Actual text 5    Qui ?
Name: french, dtype: object

-
Input text: 	wow!

Output text: ça alors !

Actual text 6    Ça alors !
Name: french, dtype: object

-
Input text: 	duck!

Output text: le chiencement.

Actual text 7    À terre !
Name: french, dtype: object

-
Input text: 	fire!

Output text: au feu !

Actual text 8    Au feu !
Name: french, dtype: object

-
Input text: 	help!

Output text: à l'aide !

Actual text 9    À l'aide !
Name: french, dtype: object


In [70]:
input_seq = encoder_input_data[seq_index: seq_index+1]
decoded_sentence = decode_sequence(input_seq)
print('\n-')
print('Input text:', eng_corpus[seq_index])
print('Output text:', decoded_sentence)
print("Actual text", preprocessed_data.french[seq_index: seq_index+1])


-
Input text: 	help!

Output text: à l'aide !

Actual text 9    À l'aide !
Name: french, dtype: object


In [71]:
preprocessed_data.head(20)

,english,french
0,Go.,Va !
1,Go.,Va !
2,Hi.,Salut !
3,Run!,Cours !
4,Run.,Cours !
5,Who?,Qui ?
6,Wow!,Ça alors !
7,Duck!,À terre !
8,Fire!,Au feu !
9,Help!,À l'aide !


In [72]:
def test():
    
    print('Enter a something in english')
    text = input()
    
    text = '\t'+text.lower()+'\n'
    test_corpus = [text]
    test_encoder_input_data = np.zeros((len(eng_corpus),max_input_length, num_encoder_tokens),dtype='float32')
    
    for i, input_text in enumerate(test_corpus):
        for j, char in enumerate(input_text):
            test_encoder_input_data[i, j, input_token_index[char]] = 1.
        test_encoder_input_data[i, j+1:, input_token_index[' ']] = 1.
            
    decoded_sentence = decode_sequence(test_encoder_input_data[:1])
    print('Output text:', decoded_sentence)
    

In [76]:
test()

Enter a something in english
hi.
Output text: salut !



In [79]:
test()

Enter a something in english
hi.....
Output text: revinez !

